In [ ]:
#optical flow 
Optical flow, nesne takibi (object tracking) uygulamalarında kullanılan önemli bir tekniktir. 
Bir video veya görüntü dizisinde hareket eden nesnelerin pikseller bazındaki hareketini takip etmek için kullanılır.
Daha spesifik olarak, ardışık iki kare arasında her bir pikselin ne kadar ve hangi yönde hareket ettiğini tespit eder.


Optical Flow'un Temel Prensibi:
Hareket Vektörleri: Optical flow algoritmaları, ardışık iki görüntüdeki her bir pikselin nasıl hareket ettiğini bulmaya çalışır. Bu hareketler, bir nesnenin hareketinin tahmin edilmesine yardımcı olan hareket vektörleri şeklinde ifade edilir.
Sürekli Hareket Varsayımı: Algoritmalar genellikle hareketin yavaş ve sürekli olduğu varsayımına dayanır, bu nedenle ani veya 
çok hızlı hareketleri tespit etmek zor olabilir.


Optical Flow Türleri:
Dense Optical Flow: Görüntüdeki her piksel için hareket vektörleri hesaplanır. Bu, daha ayrıntılı bilgi sağlasa da 
daha fazla hesaplama gücü gerektirir.


Sparse Optical Flow: Sadece belirli noktalar (örn. nesnenin köşe noktaları veya ilgi noktaları) için hareket vektörleri 
hesaplanır. Daha az hesaplama gücü gerektirir ve genellikle gerçek zamanlı uygulamalarda kullanılır.


Örnek Uygulamalar:
Lucas-Kanade Optical Flow: Yaygın kullanılan bir algoritmadır ve genellikle sparse optical flow için kullanılır. 
Bu yöntem, hareketli nesnelerin belli noktalarındaki hareketi takip etmek için kullanılır.


Farneback Optical Flow: Bu algoritma dense optical flow için kullanılır ve görüntüdeki her pikselin hareketini tahmin eder.

Kullanım Alanları:
Nesne Takibi (Object Tracking): Optical flow, bir videodaki belirli bir nesnenin veya ilginin bulunduğu bölgenin zaman
içindeki hareketini izlemek için kullanılır.
Hareket Analizi: İnsan hareketi veya araç trafiği gibi daha karmaşık hareketleri analiz etmek için kullanılabilir.
Optical flow, nesnelerin pikseller bazında hareketlerini izleyerek, nesne takibi algoritmalarında verimli ve doğru sonuçlarsağlar. 

In [5]:
import numpy as np
import cv2

# Köşe izleme parametreleri ve max köşeleri tespit
corner_track_params = dict(maxCorners=10, qualityLevel=0.05, minDistance=10, blockSize=7)
lk_params = dict(winSize=(200, 200),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Kameraya erişmeye çalışıyoruz
cap = cv2.VideoCapture(0)

# Eğer kamera açılmadıysa bir hata mesajı gösteriyoruz
if not cap.isOpened():
    print("Kamera açılamadı!")
    exit()

# İlk kareyi yakalayalım
ret, prev_frame = cap.read()

# Eğer ilk kare yakalanamadıysa
if not ret:
    print("Kare yakalanamadı!")
    cap.release()
    exit()

# Gri tonlama işlemi
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Köşeleri tespit et
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

# İlk kare ile aynı boyutta bir maske oluştur
mask = np.zeros_like(prev_frame)

while True:
    # Güncel kareyi alalım
    ret, frame = cap.read()
    if not ret:
        print("Kare yakalanamadı!")
        break

    # Gri tonlama
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Optical Flow hesaplama
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)

    # Hareket eden iyi noktaları al
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]

    # Çizgi ve daire çizme işlemleri
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()

        # Çizgi çizme
        mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_prev), int(y_prev)), (0, 255, 0), 3)

        # Daire çizme
        frame = cv2.circle(frame, (int(x_new), int(y_new)), 8, (0, 0, 255), -1)

    # Görüntü ile maskeyi birleştir ve göster
    img = cv2.add(frame, mask)
    cv2.imshow('frame', img)

    # ESC tuşuna basıldığında döngüyü kır
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Önceki kareyi ve noktaları güncelle
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)

# Pencereleri kapat ve kamerayı serbest bırak
cv2.destroyAllWindows()
cap.release()


In [ ]:
cv2.goodFeaturesToTrack fonksiyonu, görüntüde takip edilecek en iyi köşeleri bulmamızı sağlar. 
 bu köşeler, genellikle bir nesnenin keskin kenarları veya geometrik şekillerin kesişim noktaları olur.
Bu işlem için corner_track_params'ı kullanıyoruz:
maxCorners=10: Maksimum 10 köşe tespit edilir.
qualityLevel=0.3: Tespit edilen köşe noktalarının kalitesi %30 olmalıdır. Yani en iyi köşe noktalarının %30’u kabul edilir.
minDistance=7: Tespit edilen köşe noktalarının birbirine minimum 7 piksel uzaklıkta olması gerekir.
Örnek:
Bir pencere, masa kenarı gibi belirgin hatlara sahip nesneler bu noktaların tespitinde iyi örneklerdir.
Tespit edilen noktalar:
lua
Kodu kopyala
[[[150.  120.]],
 [[345.  200.]],
 [[ 30.   50.]], ... ]




mask = np.zeros_like(prev_frame)

Bu adımda, ilk görüntüyle aynı boyutlarda bir maske oluşturuyoruz. Ancak bu maske tamamen siyahtır (zeros_like), 
yani tüm piksellerin değeri 0 olur.
Neden Maske Kullanıyoruz?
Maskeyi, Optical Flow sonucu izlenen köşe noktaları arasına çizilecek çizgiler için kullanırız.
Bu çizgiler, yeni kareye değil, sadece maskeye çizilir. 
Böylece, her yeni karede bu çizgiler eklenmeye devam eder ve hareket edilen yol sürekli gösterilir.

cap.read() ile kameradan sürekli olarak kareler alıyoruz. frame değişkeninde her seferinde yeni bir kare saklanır.
ret değişkeni, eğer kare başarıyla yakalandıysa True olur, aksi durumda False olur. Eğer False dönerse, döngüyü bitiriyoruz.


Gri Tonlamaya Dönüşüm (Yeni Kare)
Yeni kareyi de gri tonlamaya çeviriyoruz (frame_gray), çünkü Optical Flow algoritması gri tonlama üzerinde çalışır.


7.3 Optical Flow Hesaplama (calcOpticalFlowPyrLK)
cv2.calcOpticalFlowPyrLK, önceki karede tespit edilen köşe noktalarının (prevPts) yeni karede nereye hareket ettiğini hesaplar.
Algoritma, her köşe noktasını bulmaya çalışır ve yeni noktaların konumlarını (nextPts), hangi noktaların başarılı şekilde takip
edildiğini (status) ve hataları (err) döner.

Örnek:
Eğer önceki karedeki bir nokta (100, 150) pozisyonundaysa ve hareket ettiyse, yeni karede bu nokta farklı bir yerde olabilir,
örneğin (105, 160).

nextPts: Yeni karedeki noktaların pozisyonları.
status: Her köşe noktası için takip başarılı mı (1) yoksa başarısız mı (0).
    
    
status dizisi, Optical Flow hesaplamasında başarılı olup olmadığını gösterir. 
Başarılı olan noktalar status == 1 olan noktalardır.

good_new: Yeni karedeki başarıyla izlenen noktalar.
good_prev: Önceki karedeki başarıyla izlenen aynı noktalar.
    

    
Koordinatları Almak
ravel(): Koordinatları düzleştirir (2 boyutlu diziyi tek boyutlu hale getirir). Örneğin, [[105.0, 160.0]] → [105.0, 160.0].
    

9.2 Çizgi Çizmek
cv2.line fonksiyonu, önceki noktadan yeni noktaya doğru yeşil bir çizgi çizer:
python
Kodu kopyala
mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_prev), int(y_prev)), (0, 255, 0), 3)
Bu çizgi, maskeye eklenir ve her karede izlenen noktaların hareketini görselleştirir.



9.3 Daire Çizmek
cv2.circle, yeni noktaların üzerine kırmızı daireler çizer:
python
Kodu kopyala
frame = cv2.circle(frame, (int(x_new), int(y_new)), 8, (0, 0, 255), -1)
Bu daireler, hareket eden köşe noktalarını daha belirgin hale getirir.

Örnek:
Diyelim ki önceki nokta (100, 150), yeni nokta (105, 160) ise:

Bu iki nokta arasında bir yeşil çizgi çizilir.
Yeni nokta (105, 160) üzerine bir kırmızı daire çizilir.

In [ ]:
#şimdi hareket edilen noktaları renklendirmek 

In [14]:
import cv2 
import numpy as np

# Capture the frame
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

# Get gray scale image of first frame and make a mask in HSV color
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    # Check out the markdown text above for a break down of these paramters, most of these are just suggested defaults
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    
    # Color the channels based on the angle of travel
    # Pay close attention to your video, the path of the direction of flow will determine color!
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # Convert back to BGR to show with imshow from cv
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # Set the Previous image as the next iamge for the loop
    prvsImg = nextImg

    
cap.release()
cv2.destroyAllWindows()

In [13]:
İlk kare ile aynı boyutlarda, fakat tüm pikselleri sıfır olan (siyah) bir HSV maskesi oluşturuyoruz.

HSV (Hue, Saturation, Value) formatı, renkleri temsil etmek için kullanılır.
Hue: Renk tonu (0-179 derece arası).
Saturation: Doygunluk (0-255 arası).
Value: Parlaklık (0-255 arası).
hsv_mask[:,:,1] = 255 ile maskenin tüm piksellerinin doygunluk değerini 255 yapıyoruz. Bu, renklerin daha canlı görünmesini sağlar.


cv2.calcOpticalFlowFarneback fonksiyonu, iki ardışık gri tonlamalı görüntü arasındaki hareketi (optical flow) hesaplar.
Parametrelerin açıklaması:
prvsImg ve nextImg: Önceki ve sonraki görüntüler.
None: Output (sonuç) verilerinin saklanacağı değişken. Bu, genellikle None olur çünkü yeni bir veri saklanır.
0.5: Ölçekleme faktörü. Bu, görüntüyü küçülterek işlemi hızlandırır.
3: Piramit katman sayısı. Görüntünün çoklu çözünürlük seviyelerinde incelenmesini sağlar.
15: Pencere boyutu. Her bir piksel için ne kadar komşu pikselin hesaba katılacağını belirtir.
3: En fazla iterasyon sayısı.
5: Kaç tane pikselin minimum hareket etmesi gerektiğini belirler.
1.2: Gaussian filtresi için sigma değeri.
0: Ek parametreler, genellikle 0.

    
Optical Flow Nedir?
Optical Flow, bir görüntüdeki piksellerin iki ardışık karede nasıl hareket ettiğini tespit eder. Bu hareketler, 
nesnelerin yer değiştirmesi veya kameranın hareketi nedeniyle oluşur. Hesaplanan "flow" değişkeni, her pikselin hareket yönünü
ve büyüklüğünü içerir.


Örnek:
Bir önceki karede (100, 150) pikselinde bir nesne varsa ve sonraki karede bu nesne (105, 155) pozisyonuna kaydıysa, optical flow
bu hareketi tespit eder ve her bir pikselin yönünü ve büyüklüğünü hesaplar.


cv2.cartToPolar: Optical flow sonuçlarını (x, y bileşenlerini) büyüklük (mag) ve açıya (ang) çevirir.
flow[:,:,0]: Optical flow'un x yönündeki bileşeni.
flow[:,:,1]: Optical flow'un y yönündeki bileşeni.
mag: Hareketin büyüklüğü (piksel bazlı hız).
ang: Hareketin yönü (derece cinsinden açı).

    
    hsv_mask[:,:,0] = ang / 2: Optical flow açısını, Hue (renk tonu) kanalına ekliyoruz. Bu sayede farklı yönlerdeki hareketler
        farklı renklerde görünecektir. ang / 2 ile açıyı 0-179 aralığına getiriyoruz (Hue değerleri bu aralıkta olmalıdır).

hsv_mask[:,:,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX): Hareketin büyüklüğünü normalleştirerek, 
    parlaklık (Value) kanalına ekliyoruz. cv2.normalize ile bu büyüklük değerlerini 0-255 aralığına çekiyoruz.

Örnek:
Eğer bir nesne sağa doğru hareket ediyorsa, açısı 0 derece olabilir ve Hue kanalı mavi renge ayarlanır.
Eğer yukarı doğru hareket ediyorsa, açı 90 derece olur ve yeşil renge ayarlanır.

SyntaxError: unterminated string literal (detected at line 35) (1063894231.py, line 35)